### Set Up

#### Standard library imports

In [1]:
import os
import datetime

#### Third party imports 

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.rnn import LSTMStateTuple

#### Local imports

In [3]:
import modules.nikhil.midi_related as midi
import modules.nikhil.batch as batch

from modules.nikhil.MyFunctions import (
    alignXy,
    Conditional_Probability_Layer,
    Input_Kernel, 
    getNumberOfBatches,
    Loss_Function_1,
    Loss_Function_2,
    LSTM_Cell,
    LSTM_Layer
)

#### Extensions and autoreload

In [4]:
%load_ext autoreload
%autoreload 2

#### Setting relative directories

In [5]:
Working_Directory = os.getcwd()
Project_Directory = os.path.abspath(os.path.join(Working_Directory,'..'))
Music_In_Directory = Project_Directory + "/data/chopin_midi/" 
Output_Directory = Project_Directory + "/outputs/"
Model_Directory = Output_Directory + "models/"
Checkpoint_Directory = Model_Directory + "ckpt/"
Checkpoint_Date_Directory = Checkpoint_Directory + "20210830/"
Checkpoint_Date_Epoch_Directory = Checkpoint_Date_Directory + "Long_Train_256"
Music_Out_Directory = Output_Directory + "midi/"
Music_Out_Genereating_Directory = Music_Out_Directory + "generated/"

#### Redefine model architecture

In [6]:
# Set parameters 
lowerBound = 21
upperBound = 109
Midi_low = lowerBound
Midi_high = upperBound - 1
num_notes = Midi_high + 1 - Midi_low # X.shape[1] = Midi_high + 1 - Midi_low 
num_timesteps = 16*3*2 
input_size = 4
keep_prob = 0.5

num_t_units = [128, 128] # [200, 200]
num_n_units = [64, 64] # [100, 100]
dense_units = 3

In [7]:
# Build the Model Graph:
tf.reset_default_graph()
print('Building Graph...')

# Graph Input Placeholders
Note_State_Batch = tf.placeholder(dtype=tf.float32, shape=[None, num_notes, None, input_size], name= "Note_State_Batch")
output_keep_prob = tf.placeholder(dtype=tf.float32, shape=(), name= "output_keep_prob")

#Generate expanded tensor from batch of note state matrices
Note_State_Expand = Input_Kernel(Note_State_Batch, 
                                 Midi_low=Midi_low, 
                                 Midi_high=Midi_high #,
                                 #time_init=time_init
                                )
Note_State_Expand_aligned, Note_State_Batch_aligned = alignXy(Note_State_Expand, Note_State_Batch)

print('Note_State_Expand shape = ', Note_State_Expand.get_shape())
print('Note_State_Batch shape = ',  Note_State_Batch.get_shape())

# Generate initial state (at t=0) placeholder
timewise_state=[]
for i in range(len(num_t_units)):
    timewise_c=tf.placeholder(dtype=tf.float32, shape=[None, num_t_units[i]]) #None = batch_size * num_notes
    timewise_h=tf.placeholder(dtype=tf.float32, shape=[None, num_t_units[i]])
    timewise_state.append(LSTMStateTuple(timewise_h, timewise_c))

timewise_state=tuple(timewise_state)

timewise_cell = LSTM_Cell(num_t_units, output_keep_prob)

timewise_out, timewise_state_out = LSTM_Layer(input_data=Note_State_Expand_aligned,
                                              state_init=timewise_state,
                                              cell = timewise_cell,
                                              time_or_note="time")

print('Time-wise output shape = ', timewise_out.get_shape())
# print('Time-wise state shape = ', timewise_state_out)

#LSTM Note Wise Graph

# Generate initial state (at n=0) placeholder
notewise_state=[]
for i in range(len(num_n_units)):
    notewise_c=tf.placeholder(dtype=tf.float32, shape=[None, num_n_units[i]]) #None = batch_size * num_timesteps
    notewise_h=tf.placeholder(dtype=tf.float32, shape=[None, num_n_units[i]])
    notewise_state.append(LSTMStateTuple(notewise_h, notewise_c))

notewise_state=tuple(notewise_state)

notewise_cell = LSTM_Cell(num_n_units, output_keep_prob)

notewise_out, notewise_state_out =  LSTM_Layer(input_data=timewise_out,
                                               state_init=notewise_state,
                                               cell=notewise_cell,
                                               time_or_note="note")

print('Note-wise output shape = ', notewise_out.get_shape())
# print('Note-wise state shape = ', notewise_state_out)

output_1, output_2, output_3 = Conditional_Probability_Layer(notewise_out, dense_units=dense_units)

print('play_articulate_logit output shape = ', output_1.get_shape())
print('velocity output shape = ', output_2.get_shape()) 
print('play_articulate_sampled output shape = ', output_3.get_shape())

print('Graph Building Complete')

Building Graph...
Note_State_Expand shape =  (?, 88, ?, 108)
Note_State_Batch shape =  (?, 88, ?, 4)
Time-wise output shape =  (?, 88, ?, 128)
Note-wise output shape =  (?, 88, ?, 64)
play_articulate_logit output shape =  (?, 88, ?, 2)
velocity output shape =  (?, 88, ?, 1)
play_articulate_sampled output shape =  (?, 88, ?, 2)
Graph Building Complete


### MIDI generation

#### Set parameters

In [24]:
primer = False #'chop6401'
n_bars = 4
batch_size_gen = 4
n_time_steps_per_sixtheenth = 3
max_sixteenth_index = 12 
n_time_steps_ber_bar = max_sixteenth_index * n_time_steps_per_sixtheenth
num_timesteps =  n_time_steps_ber_bar * n_bars 

t_gen = n_bars * n_time_steps_ber_bar
print("Create a song with", t_gen, "timesteps corresponding to", n_bars, "bars")

if primer:
    primer = midi.midiToNoteStateMatrix(Music_In_Directory + primer + ".mid", 
                                        verbose = False, 
                                        verbose_ts = False) 
    sixteenth_index = [b[0][3] for b in  primer]
    max_sixteenth_index = max(sixteenth_index)
    n_time_steps_ber_bar = max_sixteenth_index * n_time_steps_per_sixtheenth
    num_timesteps =  n_time_steps_ber_bar * (n_bars // 2) 
    batch_input_state, start_old = batch.getPieceBatch2(primer, 
                                                        num_time_steps = num_timesteps - 1, 
                                                        batch_size = batch_size_gen,
                                                        start_old = 0) 
    notes_gen_initial = batch_input_state[:,:,-(num_timesteps):,:]
    notes_gen = notes_gen_initial
    name = "primer"
else:
    num_timesteps =  n_time_steps_ber_bar * (n_bars // 2) 
    notes_gen_initial = np.zeros((batch_size_gen, num_notes, num_timesteps, 3))
    beats_initial = [int(t / n_time_steps_per_sixtheenth) % int(n_time_steps_ber_bar / n_time_steps_per_sixtheenth) + 1 for t in range(num_timesteps)]
    beats_initial = np.transpose(np.array(beats_initial, ndmin = 4), (0,1,3,2))
    beats_initial = np.tile(beats_initial, [batch_size_gen,num_notes,1,1])
    notes_gen_initial = np.concatenate([notes_gen_initial, beats_initial], axis=3)
    # Initial States
    notes_gen = notes_gen_initial
    name = "from scratch"

Create a song with 144 timesteps corresponding to 4 bars


#### Initialize

In [25]:
timewise_state_val=[]
for i in range(len(num_t_units)):
    c = np.zeros((batch_size_gen*num_notes, num_t_units[i])) #start first time step with zero state in LSTM time cells
    h = np.zeros((batch_size_gen*num_notes, num_t_units[i]))
    timewise_state_val.append(LSTMStateTuple(h, c))
        
notewise_state_val=[]
for i in range(len(num_n_units)):
    c = np.zeros((batch_size_gen*(num_timesteps_initial-1), num_n_units[i])) #start every batch with zero state in LSTM time cells
    h = np.zeros((batch_size_gen*(num_timesteps_initial-1), num_n_units[i]))
    notewise_state_val.append(LSTMStateTuple(h, c))
        
notes_gen_arr=[]

#### Genereate new MIDI files from scratch 

In [26]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    
    print("Load the model from: {}".format(Checkpoint_Date_Epoch_Directory))
    saver.restore(sess, Checkpoint_Date_Epoch_Directory)
    

    for t in range(t_gen):

        beat = int(t / n_time_steps_per_sixtheenth) % int(n_time_steps_ber_bar / n_time_steps_per_sixtheenth) + 1
        
        feed_dict = {Note_State_Batch: notes_gen[:,:,-(num_timesteps_initial):,:], 
                     timewise_state: timewise_state_val, 
                     notewise_state: notewise_state_val, 
                     output_keep_prob: keep_prob}  
        
        timewise_state_val, velocity_gen, notes_a_p, Note_State_Batch_result, Note_State_Expand_result = sess.run(
            [timewise_state_out, output_2, output_3, Note_State_Batch_aligned, Note_State_Expand_aligned],
            feed_dict = feed_dict)
        
        # new_note = np.concatenate([notes_a_p, velocity_gen, Note_State_Batch_result[:,:,:,3:4]], axis=-1)
        new_note = np.concatenate([notes_a_p[:,:,-1:,:], velocity_gen[:,:,-1:,:]], axis=-1)
        new_note_p   = new_note[:,:,:,0]
        new_note_a   = new_note[:,:,:,1] * new_note[:,:,:,0]
        new_note_vel = new_note[:,:,:,2] * new_note[:,:,:,0]
        new_note_beat = np.zeros((batch_size_gen, num_notes, 1), dtype=np.float32)
        new_note_beat.fill(beat)
        new_note = np.stack([new_note_p, new_note_a, new_note_vel, new_note_beat], axis=-1)
        notes_gen = np.concatenate([notes_gen, new_note], axis=2)
        
        if t == 0 and primer:
            current_time_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            for i in range(batch_size_gen):
                midi.generate_audio(batch_input_state[i:(i+1),:,:,:], 
                                    Music_Out_Genereating_Directory + current_time_str[:-7] + '/',
                                    'generated_batch_' + str(i) + '_primer', 
                                    verbose = False)
            prediction = np.concatenate([notes_a_p, velocity_gen, Note_State_Batch_result[:,:,:,3:4]], axis=-1)
            for i in range(batch_size_gen):
                midi.generate_audio(prediction[i:(i+1),:,:,:], 
                                    Music_Out_Genereating_Directory + current_time_str[:-7] + '/',
                                    'generated_batch_' + str(i) + '_primer_predicted', 
                                    verbose = False)
        if t == (t_gen - 3):
            current_time_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            if primer:
                for i in range(batch_size_gen):
                    midi.generate_audio(notes_gen[i:(i+1),:,:,:], 
                                        Music_Out_Genereating_Directory + current_time_str[:-7] + '/',
                                        'generated_batch_' + str(i) + '_from_primer', 
                                        verbose = False)
            else:
                for i in range(batch_size_gen):
                    midi.generate_audio(notes_gen[i:(i+1),:,:,:], 
                                        Music_Out_Genereating_Directory + current_time_str[:-7] + '/',
                                        'generated_batch_' + str(i) + '_from_scratch', 
                                        verbose = False)
            

Load the model from: /home/mirko/Documents/FHWN/MA/master_thesis/code/tf1/outputs/models/ckpt/20210830/Long_Train_256
INFO:tensorflow:Restoring parameters from /home/mirko/Documents/FHWN/MA/master_thesis/code/tf1/outputs/models/ckpt/20210830/Long_Train_256
